In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
yr = 2016
run_no = 1
rundays = 365
runhours = rundays*24

hr_range = pd.date_range('1/1/'+str(yr)+'', periods=runhours, freq='H')
day_range = pd.date_range('1/1/'+str(yr)+'', periods=rundays, freq='D')

In [3]:
line_data = pd.read_csv('data_camb_transparam.csv',header=0)
vlt_angle = pd.read_csv('out_camb_R'+str(run_no)+'_'+str(yr)+'_vlt_angle.csv',header=0)

In [4]:
####======== Lists of Nodes of the Power System ========########
h_nodes = ['TTYh','LRCh','ATYh','KIR1h','KIR3h','KMCh']
h_imports = ['Salabam']
##s_nodes = ['solar1','solar2']
##w_nodes =['wind1','wind2']

gd_nodes = ['GS1','GS2','GS3','GS5','GS7','KPCM','KPT','SHV','SRP'] ##Thermoplant nodes with demand
gn_nodes = ['STH','Thai','Viet'] ##Thermoplants nodes without demand

g_nodes = gd_nodes + gn_nodes
##print ('Gen_Nodes:',len(g_nodes))

td_nodes = ['GS4','GS6','BTB','BMC','STR','TKO','KPS'] ##Transformers with demand
tn_nodes = ['IE','KPCG','OSM','PRST'] ##Transformers without demand

d_nodes = gd_nodes + td_nodes
##print ('Demand_Nodes:',len(d_nodes))

all_nodes = h_nodes + h_imports + gn_nodes + gd_nodes + tn_nodes + td_nodes ## + s_nodes + w_nodes
##print ('Total_Nodes:',len(all_nodes))

indx = 0

trans_data = [[] for n in range(4)]
trans_mwh = []
trans_usage = []

import warnings
warnings.filterwarnings('ignore')

for x in all_nodes:
    for y in all_nodes:
        if (line_data.loc[(line_data.source==x) & (line_data.sink==y),'linesus'].values) > 0: ##
            indx += 1
            
            sus = line_data.loc[(line_data.source==x) & (line_data.sink==y),'linesus'].values ##single value
            cap = line_data.loc[(line_data.source==x) & (line_data.sink==y),'linemva'].values ##single value
            
            vlt_angle_x = vlt_angle.loc[vlt_angle['Node']==x,'Value'].values ##all hourly values 
            vlt_angle_y = vlt_angle.loc[vlt_angle['Node']==y,'Value'].values ##all hourly values 
            
            vlt_angle_diff = vlt_angle_x - vlt_angle_y
            
            line_trans = vlt_angle_diff * sus
            line_usage = abs(line_trans)*100/cap ##percentage of line_usage
            
            trans_data[0].append(x)
            trans_data[1].append(y)
            
            trans_data[2].append(sus[0])
            trans_data[3].append(cap[0])           
            
            trans_mwh.append(line_trans)
            trans_usage.append(line_usage) 

            #print (indx, x, y,sus)           

len(trans_mwh)

32

In [5]:
### Yearly mean of trans & usage
trans_mean = np.mean(trans_mwh,axis=1)
usage_mean = np.mean(trans_usage,axis=1)


# N-1 Violations

In [6]:
### Hourly n-1 violations
n1_hourly = np.zeros((len(trans_usage),len(trans_usage[0])))

for i in range(len(trans_usage)):
    for j in range(len(trans_usage[0])):
        if trans_usage[i][j]>=75:
            n1_hourly[i][j] += 1

##TS of hourly n-1 violations
ts_n1_hourly = []
for i in range(len(trans_usage)):
    ts_n1_hourly.append(pd.Series(n1_hourly[i], index=hr_range))

##Monthly n-1 violations
n1_monthly = []
for i in range (len(trans_usage)):    
    n1_monthly.append((ts_n1_hourly[i].resample('M').sum()).values)

##Annual n-1 violations
n1_yearly = []
for i in range (len(trans_usage)):
    n1_yearly.append(sum(n1_hourly[i])) 

In [7]:
yearly_trans = [trans_data[0],trans_data[1],trans_data[2],trans_data[3],usage_mean,n1_yearly]
cols = ['source','sink','linesus','linemva','avg_line_usage (%)','n1_yearly']

df_yearly_trans = pd.DataFrame(yearly_trans).transpose()
df_yearly_trans.columns = cols

df_yearly_trans

,source,sink,linesus,linemva,avg_line_usage (%),n1_yearly
0,LRCh,TTYh,0.071,1722,5.64185,0
1,LRCh,OSM,0.065,1208,16.8824,0
2,ATYh,OSM,0.17,1722,1.54283,0
3,KIR1h,KIR3h,0.084,170,5.50366,0
4,KIR1h,KPS,0.041,170,8.77678,0
5,KMCh,KPT,0.228,604,8.02951,0
6,Salabam,STR,0.305,170,2.57978,0
7,Thai,IE,0.733,604,2.06189,0
8,Viet,TKO,0.05,604,15.1835,0
9,GS1,GS3,0.256,238,10.9572,0


In [8]:
###df_yearly_trans.to_csv('Result_R'+str(run_no)+'_'+str(yr)+'_trans_usage_n1.csv')